# LIGN
Recurrent Graph Neural Network for Lifelong Learning

----

## Imports

In [1]:
import lign as lg
import lign.utils.io as io
import torch as th
import torchvision as tv
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

----

## Preprocessing 
The data needs to formatted and set up for lign. A GraphDatabase class is needed to load graphs into the model and track the node. **_Only needs to be ran once_**

### Create Dataset
*_ Only run one of the following cells_

In [2]:
dataset = io.cifar_to_lign("data/datasets/CIFAR100")
dataset.save("data/cifar100.lign")

### Load Dataset
*_ Only run one of the following cells_

In [3]:
print(len(dataset))

50000


In [4]:
dataset = lg.graph.GraphDataset("data/cifar100.lign")

### Functions and NNs
Functions and NNs used throughout the code to apply changes to the graph's data

In [ ]:
def sum_neighs_data(neighs):
    out = neighs[0]
    for neigh in neighs[1:]:
        out = out + neighs
    return out

class G_Linear(nn.Module):
    def __init__(self, in_feats, out_feats, activation):
        super(NodeApplyModule, self).__init__()
        self.linear = nn.Linear(in_feats, out_feats)
        self.activation = activation

    def forward(self, x):
        x = self.linear(x)
        if self.activation:
            x = self.activation(x)
        return x

---

## Model